
# Kubernetes Lab

This lab is presented in several sections

0. [Setup](0.SETUP.ipynb)
1. [Concepts](1.Concepts.ipynb)
2. [Kubectl Basics](2.kubectl_basics.ipynb)
3. [Running pods, deployments](3.pods.ipynb)
4. [Exposing Services](4.services.ipynb)
5. [Maintenance Operations](5.maintenance.ipynb)
6. [Tools](6.tools.ipynb)

## Contributions

Your contributions to this tutorial are welcome, either via

- Opening issues on github at [ContainerOrchestration/Labs](https://github.com/ContainerOrchestration/Labs)
- Submitting PRs
- tweets to @mjbright
